# Modélisation Pile à combustible (PAC)

Auteur : Nicolas Monrolin

Date : 08/04/2021

Ces notes reprennent une partie des informations échangée lors de la réunion de lancement du stage de Thomas Ligeois le 02/04/2021 en présence de Xavier Roboam, Amine Jaafar et Christophe Turpin (LAPLACE). 

## 1) Contraintes de fonctionnement des PAC

Il existe plusieurs technologies de piles à combustibles avec des températures de fonctionnement différentes. Nous citerons ici que deux d'entre elles avec des températures de fonctionnement différentes:

* les Proton Exchange Membrane Fuel Cell (PEMFC) (entre 50-100 °C)

* les Solid Oxyd Fuel Cell (SOFC) (700-1000°C)

Chaque technologie a ses avantages et inconvénients voir tableau ci dessous, tiré d'un article de 2014 sur la cogénération (produciton d'éléctrictié et/ou de chaleur) de [Thierry Priem](https://www-techniques-ingenieur-fr.gorgone.univ-toulouse.fr/res/pdf/encyclopedia/42494210-d3360.pdf):

![](comparaison.PNG)

Pour plus d'informations, la revue technologique suivante peut constituer un point de départ [Kirubakaran2009](https://linkinghub.elsevier.com/retrieve/pii/S1364032109000872).
Contraintes importantes:

* Le stack (partie de la pile où O2 et H2 sont "mis en contact") doit avoir une température bien déterminé et HOMOGENE sur toute sa surface. On s'autorise une variation de température du fluide caloporteur de 5°C max (pour la techno PEMFC en tout cas, à vérifier pour les SOFC).

* Pour alimenter la pile en oxygène on prélève ce dernier dans l'air ambiant, avec un efficacité assez variable. On compte en général $\lambda_{air}=3$, c'est à dire qu'il faut prélever trois fois plus d'air (en quantité molaire) que ce qui est théoriquement nécessaire pour alimenter la pile en O2.

* La réaction au sein de la pile est la suivante
$$ 2H_2 + O_2 = 2H_2O$$

## 2) Modélisation

### a) Les paramètres d'entrée

* $P$ : la puissance de design de la pile. Elle peut être différente de la puissance d'utilisation en opération : on peut atteindre un meilleur rendement lorsque la pile fonctionne à une puissance inférieure à sa puissance de design.

* $U_c$ : la tension d'une seule cellule (paire d'électrodes séparées par l'électrolyte). En générale **cette tension est fixée et approche 0.7 V**. La tension *totale* de la pile pourra varier selon que les cellules seront placées en série (on somme les tensions) ou en parallèle (on somme les intensités). Une pile peut donc être consue pour délivrer une certaine tension et/ou un certain courant en fonction du nombre de cellule placées en série et/ou en parallèle.

* $\lambda_{air}$ :  ce que j'appelle l'efficacité d'extraction de l'oxygène. Pour obtenir une certaine quantité de  dioxygène on est obligé de prélever environ trois fois plus d'air que ce qui est théoriquement nécessaire.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *

# INPUTS
P = 150e3 # (W) design power
Uc = 0.7  # (V) nominal voltage of one cell
lambda_air = 3 # efficiency of oxygen extraction

### b) Calcul des flux

* $I_{tot} = N I_c = P/U_c$ (A) la somme des courants traversant toutes les cellules ($N$ nombre de cellules, $I_c$ intensité dans une seule cellule). 

> **ATTENTION**:
$I_{tot}$ n'est pas forcément égal au courant délivré par la pile $I$. Si toutes les cellules sont placées en série, la pile délivre ou plus l'instensité pouvant traverser une seule celulle $I=I_c$ et une tension $U=N U_c$. Si toutes les cellules sont en parralèles alors $I =  I_{tot} = N I_c$ et $U = U_c$.

* $J_{H2}$ (mol/s) le débit molaire d'hydrogène pour alimenter toutes les cellules

* $J_{O2}$ (mol/s) le débit molaire d'oxygène pour alimenter toutes les cellules.

On déduit les débits molaire à partir du courant $I_c$ (C/s), de la constante de Faraday $\mathcal{F}=e \mathcal{N}_A = 96485$ C/mol (où $e$ désigne la charge élémentaire (C) et $\mathcal{N}_A$ la constante d'Avogadro) et à partir du fait que le dihydrogène $H_2$ est bivalent (deux électrons de valence dans la molécule). Chaque noyau d'hydrogène pour passer la membrane doit se séparer d'un électron.
$$ J_{H_2} = \frac{I_{tot}}{2\mathcal{F}}$$

On peut ensuite en déduire le débit de dioxygène d'après l'[équation de la réaction](eq:reaction):
$$ J_{O_2} = \frac{1}{2}J_{H_2}$$

Les débits massiques se déduisant à partir des masses molaires:

* $\dot{m}_{H_2} = \mathcal{M}_{H_2}J_{H_2}$ avec $\mathcal{M}_{H_2}=2.016$ g/mol.

* $\dot{m}_{O_2} = \mathcal{M}_{O_2}J_{O_2}$ avec $\mathcal{M}_{O_2}=31.999$ g/mol.


In [9]:
# Flux
I_tot = P/Uc            # (A) Cell current
J_H2 = I_tot/(2*96485.) # (mol/s)
J_O2 = J_H2/2        # (mol/s)

print("Ic = %0.3e A" %I_tot)
print("J_H2 = %0.3f mol/s  = %0.3f g/s" %(J_H2, J_H2*2.016))
print("J_O2 = %0.3f mol/s  = %0.3f g/s" %(J_O2, J_O2*31.999))

Ic = 2.143e+05 A
J_H2 = 1.110 mol/s  = 2.239 g/s
J_O2 = 0.555 mol/s  = 17.767 g/s


**Le débit d'air à prélever** (et à compresser si l'on se trouve en altitude pour atteinde une pression de fonctionnement de la pile l'ordre de 1 bar) se déduit à partir de l'efficacité de prélèvement de dioxygène:

$$ \dot{m}_{air, prélevé} = \lambda_{air}\times \dot{m}_{air,théorique}$$

Le débit d'air théorique se calcule à partir de la fraction molaire d'$O_2$ dans l'air sec. On fera dans un premier temps l'hypothèse d'**AIR SEC** (absence de vapeur d'eau):

Molécule | fraction molaire | Masse Molaire (g/mol)
-|:-:|:-:
$N_2$ | 0.78 | 28.013
$O_2$ | 0.21 | 31.999
Autre | 0.01 | -
Air   | 1    | 28.965

$$ \dot{m}_{air,théorique} = \mathcal{M}_{air} J_{air,théorique} = \mathcal{M}_{air} \frac{J_{O_2}}{0.21} $$

> Note
Il y a une coquille dans le fichier excel original car la fraction molaire est appliquée au débit massique (et non au débit molaire). D'où la différence avec notre résultat.

In [3]:
J_air = lambda_air * J_O2 / 0.21
print("J_air = %0.3f mol/s = %0.3f g/s" %(J_air,28.965*J_air))

J_air = 7.932 mol/s = 229.747 g/s


## Travail de compression du flux d'air

La PAC est alimentée par un flux d'air qui doit être amené à une pression (totale ?) proche de 1.5 bar (A VERIFIER). D'après le paragraphe précédent on sait calculer ce flux d'air en fonction de la puissance $P$ délivrée par la pile:

$$\dot{m}_{air} = \lambda_{air}\frac{\mathcal{M}_{air}}{0.21}\frac{I_{tot}}{4\mathcal{F}} = \lambda_{air}\frac{\mathcal{M}_{air}}{0.21}\frac{P}{4\mathcal{F}U_c}$$

En vol de croisière à haute altitude (11000m), la pression extérieure est d'environ 226 hPa (0.2 bar) ce qui implique de comprimer l'air avec un taux de compression de l'ordre de 5 et qui dépend du Mach de vol.

In [4]:
from atmosphere_isa import T_ISA, p_ISA, rho_ISA, a_ISA, kts_ms, ft_m
h = 11000.33333
mach = 0.8
pt1 = p_ISA(h)*(1+0.4/1.4*mach**2)**(1.4/0.4) # inlet total pressure
pt2 = 1.5e5                                   # desired outlet pressure (recquiered at the inlet of the Fuel Cell)
print(f"Altitude {ft_m(h):.0f} ft, pression {p_ISA(h):.0f} Pa, mach {mach:.2f}")
print(f"Rapport de pressions totales pt2/pt1 = {pt2/pt1:.1f}")

Altitude 36090 ft, pression 22631 Pa, mach 0.80
Rapport de pressions totales pt2/pt1 = 3.7


Pour estimer le taux de compression $\pi_c = p_{t2}/p_{t1}$, on considère que $p_{t2}=1.5 bar$ et que $p_{t1} = p_1\left(1+\frac{\gamma-1}{\gamma} M^2\right)^\frac{\gamma}{\gamma-1}$ à l'entrée d'air, en négligeant les pertes de charge dans cette dernière:

$$\pi_c = \frac{p_{t2}}{p_1 \left(1+\frac{\gamma-1}{\gamma} M^2\right)^\frac{\gamma}{\gamma-1} }$$

La dépendance du taux de compression en Mach et en altitude est illustrée dans le tableau suivant :

Altitude (m) | Mach de vol | Taux de Compression $\pi_c$
-|:-:|:-:
11000 | 0.8 | 3.7
11000 | 0.5 | 5.2
6000  | 0.8 | 1.7
6000  | 0.5 | 2.5
0     | 0.5 | 1.2

Calculons à présent la puissance de compression isentropique $P_c$ à fournir. De manière générale, le travail de compression d'une masse $m$ de gaz s'écrit comme la variation d'enthalpie $W_c = \Delta h = m c_p (T_2-T_1)$ où $T_1$ et $T_2$ désignent la température avant et après compression respectivement. Dans le cas où l'énergie cinétique du fluide varie, comme pour de l'air prélevé à bord d'un avion, nous préférerons l'expression plus générale du travail avec l'enthalpie totale $h_t$

$$W_c = \Delta h_t = m c_p (T_{t2}-T_{t1})$$

où $T_t = T + \frac{V^2}{2c_p}$ est la température totale de l'air. Pour une transformation adiabatique réversible

$$\frac{p_{t2}}{p_{t1}} = \left(\frac{T_{t2}}{T_{t1}}\right)^\frac{\gamma}{\gamma-1}$$

et pour un rapport de compression $\pi_c = p_{t2}/p_{t1}$ on obtient

$$W_c = m c_p T_{t1}\left(\pi_c^\frac{\gamma-1}{\gamma}-1\right).$$

En pratique, pour des taux de compression autour de 2, on utilisera (probablement ?) un compresseur centrifuge dont le rendement isentropique maximal atteint environ $\eta_{is} = 80\%$. On peut espérer atteindre $85\%$ si le compresseur est optimisé pour un seul point de fonctionnement, mais au risque que ce dernier n'assure pas tous les points de vol.
La puissance réelle à fournir sera donc:

$$P_c = \frac{\dot{W}_c}{\eta_{is}} = \frac{\dot{m}_{air} c_p T_{t1}}{\eta_{is}}\left(\pi_c^\frac{\gamma-1}{\gamma}-1\right) $$

Ratio entre puissance consommée par le compresseur $P_c$ et puissance délivrée par la pile $P$ :
$$P_c/P =\frac{\lambda_{air}}{\eta_{is}}\frac{\mathcal{M}_{air}}{0.21}\frac{1}{4\mathcal{F}U_c}c_p T_{t1}\left(\pi_c^\frac{\gamma-1}{\gamma}-1\right)$$

D'après la courbe suivante, il apparaît que la puissance à fournir pour la compression de l'air peut atteindre 20% de la puissance de la PAC à haute altitude. De plus, **un taux de compression de 5 nécessite deux compresseurs centrifuges en série, ce qui diminue encore le rendement isentropique global** (notion de rendement polytropique ???)

In [5]:
mach_slider = FloatSlider(0.5, min=0, max=0.9, step=0.1, description=r'\(mach\)', continuous_update=False)
p_t2_slider = IntSlider(1.5e5, min=1e5, max=2e5, step=0.1e5, description=r'\(p_{t2}\)', continuous_update=False)
eta_is_slider = FloatSlider(0.8, min=0.60, max=0.85, step=0.05, description=r'\(\eta_{is}\)', continuous_update=False)
lambda_air_slider = FloatSlider(3, min=1, max=4, step=0.5, description=r'\(\lambda_{air}\)', continuous_update=False)
disa_slider = IntSlider(0, min=-20, max=25, step=5, description=r'\(disa\)', continuous_update=False)

In [6]:
def compressor_power_ratio(h,mach,pt2,eta_is,lambda_air,disa):
    """Compressor power ratio Pc/P
    h : altitude (m),
    mach : airplane cruise mach number
    pt_in : total pressure (Pa) recquired at the input of the fuel cell"""
    pt1 = p_ISA(h)*(1+0.4/1.4*mach**2)**(1.4/0.4) # total pressure before compressor
    pic = pt2/pt1 # pressure ratio
    cp = 1004 # J/kg/K for air
    Tt1 = T_ISA(h)+disa+(a_ISA(h)*mach)**2/(2*cp)
    Tt2 = Tt1*(pt2/pt1)**(0.4/1.4)
    return lambda_air/eta_is*(28.965e-3/(0.21*4*96485*Uc))*cp*Tt1*(pic**(0.4/1.4)-1),pic,Tt2

def plot_over_h(mach=0.5, pt2=1.5e5,eta_is=0.8,lambda_air=3,disa=0):
    h = np.linspace(0,12000,1000)
    cpr,pic,Tt2 = compressor_power_ratio(h,mach,pt2,eta_is,lambda_air,disa)
    
    fig, axs = plt.subplots(1, 3, sharey=True)
    axs[0].plot(cpr,h, 'y')
    axs[0].set_ylabel('A définir')
    axs[0].set_title(r"$P_c/P$")
    axs[1].plot(pic/10,h,label=r"$\pi_c/10$")
    axs[1].set_title(r"$\pi_c/10$")
    axs[2].plot(Tt2-273.15,h,'--k')
    axs[2].set_title("T exhaust")
    
    for ax in axs.flat:
        ax.grid()
        ax.set_xlabel("T (°C)")
    
   
    """
    plt.plot(cpr,h,label=r"$P_c/P$")
    plt.plot(pic/10,h,label=r"$\pi_c/10$")
    plt.legend()
    plt.xlim([0,0.5])
    plt.grid()
    
    ax = plot.twiny()
    ax.plot(Tt2-273.15,h,'--k',label="T exhaust")
    ax.xlim([20,150])
    ax.legend(loc=4)
    
    """


In [7]:
slider = widgets.VBox([mach_slider, p_t2_slider, eta_is_slider, lambda_air_slider, disa_slider])
out = widgets.interactive_output(plot_over_h, {'mach': mach_slider, 'pt2' : p_t2_slider, 'eta_is':eta_is_slider, 
                                               'lambda_air':lambda_air_slider, 'disa':disa_slider})
widgets.AppLayout(header=None, left_sidebar=slider, right_sidebar=out, footer=None)

AppLayout(children=(VBox(children=(FloatSlider(value=0.5, continuous_update=False, description='\\(mach\\)', m…

## Rendements

TODO : D'après le livre de [M. Boudellal 2012](https://catalogue-archipel.univ-toulouse.fr/permalink/f/f28tl6/33TOUL_ALMA51445804370004116)

